In [ ]:
import os
import numpy as np
import pandas as pd
import statistics
from matplotlib import pyplot as plt

In [ ]:
input_version = 5
codebook_version = 2
output_version = 2

output_flag = True

input_file = "derived-dataframes/regression-data-v{}/codebook{}_longform.csv".format(input_version, codebook_version)
output_dir = "output/figures-v{}".format(output_version)

if output_flag:
    try:
        os.mkdir(output_dir)
    except FileExistsError:
        print("High-level output directory already exists; no action taken.")

In [ ]:
# util for displaying dataframes
# the defaults are actually 60 & 20, but that gets annoying
def show(da, rows = 20, cols = 20, width = None):
    pd.set_option("display.max_rows", rows)
    pd.set_option("display.max_columns", cols)
    pd.set_option("display.max_colwidth", width)
    display(da)
    pd.reset_option("max_rows")
    pd.reset_option("max_columns")
    pd.reset_option("display.max_colwidth")

In [ ]:
da1 = pd.read_csv(input_file, keep_default_na=False, na_values=["_"])
da1

In [ ]:
da1["code"].value_counts().sort_index()

## Get request types by majority vote across annotators per interaction

In [ ]:
da1.groupby(by=["document", "conversation_number"]).aggregate({"voted_conversation_requests" : "first"}).value_counts()

In [ ]:
(57 + 55) / (57 + 55 + 15 + 6)

### Measure dissent

In [ ]:
# one entry per conversation-annotator
da2_condensed = da1[da1["code"].str.startswith("Big picture of an interaction > resolveRequest")]
da2_condensed.shape

In [ ]:
requests_agree = da2_condensed["conversation_requests"] == da2_condensed["voted_conversation_requests"]
requests_agree.value_counts()

In [ ]:
da2_condensed.loc[~requests_agree]["voted_conversation_requests"].value_counts()

In [ ]:
a = da2_condensed.loc[~requests_agree]["voted_conversation_requests"].tolist()
b = da2_condensed.loc[~requests_agree]["conversation_requests"].tolist()
vals, counts = np.unique(list(zip(a, b)), return_counts=True, axis=0)

print("MAJORITY         CLAIMED                         NO. INSTANCES")
for i in range(len(vals)):
    maj = vals[i][0]
    cla = vals[i][1]
    print(maj, " " * (15 - len(maj)), cla, " " * (30 - len(cla)), counts[i])

## Get outcomes by majority vote across annotators per interaction

In [ ]:
da1.groupby(by=["document", "conversation_number"]).aggregate({"voted_conversation_outcome" : "first"}).value_counts()

### Measure dissent

In [ ]:
outcomes_agree = da2_condensed["conversation_outcome"] == da2_condensed["voted_conversation_outcome"]
outcomes_agree.value_counts()

In [ ]:
da2_condensed.loc[~outcomes_agree]["voted_conversation_outcome"].value_counts()

In [ ]:
a = da2_condensed.loc[~outcomes_agree]["voted_conversation_outcome"].tolist()
b = da2_condensed.loc[~outcomes_agree]["conversation_outcome"].tolist()
c = da2_condensed.loc[~outcomes_agree]["voted_conversation_requests"].tolist()
vals, counts = np.unique(list(zip(a, b, c)), return_counts=True, axis=0)

print("REQUESTS              MAJORITY    CLAIMED     NO. INSTANCES")
for i in range(len(vals)):
    maj = vals[i][0]
    cla = vals[i][1]
    req = vals[i][2]
    print(req, " " * (20 - len(req)), maj, " " * (10 - len(maj)), cla, " " * (10 - len(cla)), counts[i])

In [ ]:
# interestingly, most disagreement on outcome occurs in codeWrite requests

## Get outcomes per request type

In [ ]:
convda = da1.groupby(by=["document", "conversation_number"])
convda = convda.aggregate({"voted_conversation_requests" : "first", 
                           "voted_conversation_outcome" : "first"})
convda

In [ ]:
convda["success"] = convda["voted_conversation_outcome"] == "S"

In [ ]:
request_outcomes_da = convda.groupby(by=["voted_conversation_requests"]).aggregate({"success" : ["sum", "count"]})
request_outcomes_da[("success", "rate")] = request_outcomes_da[("success", "sum")] / request_outcomes_da[("success", "count")]
request_outcomes_da.sort_values(("success", "count"), ascending=False)

## Content domain frequencies

In [ ]:
# this should eventually move into the data preprocessing script
if codebook_version == 1:
    da1["code_contentDomain"] = np.where(da1["code"].str.startswith("General message attributes > contentDomain"), 
                                         da1["code"].str.split(" > ").str[2], 
                                         "N/A")

# get the relevant rows
conda = da1[da1["code_contentDomain"] != "N/A"].copy()

# shorten some variable names to make the labels on the graph a little easier to look at
if codebook_version in {1, 2, 3}:
    conda["code_contentDomain"] = conda["code_contentDomain"].replace(
        {"proposedNewCode" : "newCode", 
         "codeSpecifications" : "specifications"}) 
if codebook_version in {4}:
    conda["code_contentDomain"] = conda["code_contentDomain"].replace(
        {"higherLevelInstruction" : "holisticHelp", 
         "rapportBuilding" : "rapport", 
         "codeSpecifications" : "specifications"})

# compute the thing
def tmp(da):
    da = da["code_contentDomain"].value_counts()
    for c in conda["code_contentDomain"].unique():
        if not c in da.index:
            da[c] = 0
    return da.sort_index()

confr = conda.groupby(by="annotator").apply(tmp)
confr = confr.median(axis=0).sort_values(ascending=False)

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.bar(confr.index, confr)
fl, ft, fa = 18, 20, 20
plt.xlabel("Content domain", fontsize=fa)
plt.ylabel("Number of instances", fontsize=fa)
plt.title("Content domain frequencies across all data", fontsize=ft)
plt.xticks(rotation=50, ha="right", fontsize=fl)
fig.show()
if output_flag:
    fig.savefig(os.path.join(output_dir, "content-counts.png"), bbox_inches = "tight")

### Do it again but per-request-type

#### Show them individually

In [ ]:
confr = conda.groupby(by=["voted_conversation_requests", "annotator"]).apply(tmp)
confr = confr.groupby(by="voted_conversation_requests").aggregate("median")
confr

In [ ]:
for request in confr.index:
    fr = confr.loc[request]
    
    fig = plt.figure(figsize=(12, 4))
    plt.bar(fr.index, fr)
    fl, ft, fa = 18, 20, 20
    plt.xlabel("Content domain", fontsize=fa)
    plt.ylabel("Number of instances", fontsize=fa)
    plt.title("Content domain frequencies for request type {}".format(request), fontsize=ft)
    plt.xticks(rotation=50, ha="right", fontsize=fl)
    fig.show()
    if output_flag:
        fig.savefig(os.path.join(output_dir, "{}-content-counts.png".format(request)), bbox_inches = "tight")

#### Show them all at once

In [ ]:
confrp = confr.div(confr.sum(axis=1), axis=0)
confrp = confrp.rename(columns={"bug" : "Bug", 
                                "codeOpinion" : "Code opinion", 
                                "codingConcept" : "Coding concept", 
                                "codingExperience": "Coding experience", 
                                "developmentStrategy": "Development strategy", 
                                "errorLocation": "Error location", 
                                "errorMsg": "Error message", 
                                "learningResources": "Learning resources", 
                                "newCode": "Proposed new code", 
                                "originalCode": "Original code", 
                                "personalInfo": "Personal information", 
                                "platformRelated": "Platform related", 
                                "specifications": "Specifications", 
                                "testCases": "Test cases"})
confrp = confrp.rename(index={"bugFix" : "Bug fix", 
                              "codeComprehension" : "Code comprehension", 
                              "codeImprove" : "Code improve", 
                              "codeWrite" : "Code write"})

In [ ]:
confrp = confrp[confrp.sum().sort_values(ascending=True).index]
confrp = confrp.transpose()
confrp

In [ ]:
ax = confrp.plot(kind='bar', figsize=(12, 4), width=0.8)
ax.set_xlabel("Content domain", fontsize=fa)
ax.set_ylabel("Proportion of instances", fontsize=fa)
ax.set_title("Content domain frequencies broken down by request type", fontsize=ft)
ax.legend(fontsize=fl-2)
plt.grid()
plt.xticks(rotation=30, ha="right", fontsize=fl-2)
plt.yticks(fontsize=fl-2)
if output_flag:
    plt.savefig(os.path.join(output_dir, "request-content-counts.pdf"), bbox_inches = "tight")
else:
    plt.show()

### Do it again but only for Learner questions

In [ ]:
# FIXME I like this better, need to implement it everywhere
#confr = conda.groupby(by=["document", "conversation_number", "voted_conversation_requests", "annotator"]).apply(tmp)
#confr = confr.groupby(by=["document", "conversation_number", "voted_conversation_requests"]).aggregate("median")
#confr = confr.groupby(by="voted_conversation_requests").aggregate("sum")
#confr

In [ ]:
assert(codebook_version in {2, 3, 4}) # need this structure for this to work

qconda = conda[(conda["code_primary"] == "Questioning") & conda["speakerIsLearner"]] # & (conda["conversation_strict"] == True)
qconfr = qconda.groupby(by=["document", "conversation_number", "voted_conversation_requests", "annotator"]).apply(tmp)
qconfr = qconfr.groupby(by=["document", "conversation_number", "voted_conversation_requests"]).aggregate("median")
qconfr = qconfr.groupby(by="voted_conversation_requests").aggregate("sum")
qconfr

In [ ]:
assert(codebook_version in {2, 3, 4})

qconfrp = qconfr.div(qconfr.sum(axis=1), axis=0) # row-normalize

In [ ]:
assert(codebook_version in {2, 3, 4})

ax = qconfrp.transpose().plot(kind='bar', figsize=(12, 4), width=0.8)
ax.set_xlabel("Content domain", fontsize=fa)
ax.set_ylabel("Proportion of instances", fontsize=fa)
ax.set_title("Learner question content frequencies broken down by request type", fontsize=ft)
ax.legend(fontsize=fl-2)
plt.xticks(rotation=50, ha="right", fontsize=fl)
if output_flag:
    plt.savefig(os.path.join(output_dir, "request-learner-question-content-counts.png"), bbox_inches = "tight")
else:
    plt.show()

### Do it again but only for Helper explanations/help

In [ ]:
assert(codebook_version in {2, 3, 4}) # need this structure for this to work

hconda = conda[(conda["code_primary"] == "Helping") & ~conda["speakerIsLearner"]] # & (conda["conversation_strict"] == True)
hconfr = hconda.groupby(by=["document", "conversation_number", "voted_conversation_requests", "annotator"]).apply(tmp)
hconfr = hconfr.groupby(by=["document", "conversation_number", "voted_conversation_requests"]).aggregate("median")
hconfr = hconfr.groupby(by="voted_conversation_requests").aggregate("sum")
hconfr

In [ ]:
assert(codebook_version in {2, 3, 4})

hconfrp = hconfr.div(hconfr.sum(axis=1), axis=0) # row-normalize

In [ ]:
assert(codebook_version in {2, 3, 4})

ax = hconfrp.transpose().plot(kind='bar', figsize=(12, 4), width=0.8)
ax.set_xlabel("Content domain", fontsize=fa)
ax.set_ylabel("Proportion of instances", fontsize=fa)
ax.set_title("Helping content frequencies broken down by request type", fontsize=ft)
ax.legend(fontsize=fl-2)
plt.xticks(rotation=50, ha="right", fontsize=fl)
if output_flag:
    plt.savefig(os.path.join(output_dir, "request-helper-helping-content-counts.png"), bbox_inches = "tight")
else:
    plt.show()

## Experience & personal info

### Learners

#### Coding experience

This doesn't work for codebook 4, because it's been pooled with other things!

In [ ]:
assert(not codebook_version in {4})

expda = da1[(da1["code_contentDomain"] == "codingExperience") & da1["speakerIsLearner"]]
expda = expda.sort_values(["document", "conversation_number", 
                           "quote_startPosition", "quote_endPosition", "annotator"])
expda = expda[["document", "conversation_number", "annotator", "quote_text", 
               "quote_startPosition", "quote_endPosition"]]

pd.set_option("display.max_colwidth", None)
expda.groupby(by=["document"]).apply(display)
pd.reset_option("display.max_colwidth")

In [ ]:
assert(not codebook_version in {4})

# "we saw everything in 1 week"
pd.set_option("display.max_colwidth", None)
display(da1[(da1["document"] == "DRskphqiwF.txt") & 
            (da1["conversation_number"] == 0) & 
            (da1["annotator"] == "A") &
            (5200 < da1["quote_startPosition"]) & 
            (da1["quote_startPosition"] < 5400)])
pd.reset_option("display.max_colwidth")

In [ ]:
assert(not codebook_version in {4})

# "yes, beginning though"
pd.set_option("display.max_colwidth", None)
display(da1[(da1["document"] == "6SdCx2rR9F.txt") & 
            (da1["conversation_number"] == 0) & 
            (3500 < da1["quote_startPosition"]) & 
            (da1["quote_startPosition"] < 3707)])
pd.reset_option("display.max_colwidth")

#### Personal info

In [ ]:
assert(not codebook_version in {4})

perda = da1[(da1["code_contentDomain"] == "personalInfo") & da1["speakerIsLearner"]]
perda = perda.sort_values(["document", "conversation_number", 
                           "quote_startPosition", "quote_endPosition", "annotator"])
perda = perda[["document", "conversation_number", "annotator", "quote_text", 
               "quote_startPosition", "quote_endPosition"]]

pd.set_option("display.max_colwidth", None)
perda.groupby(by=["document"]).apply(display)
pd.reset_option("display.max_colwidth")

In [ ]:
assert(not codebook_version in {4})

# nontraditional 1 (3 hr drive)

In [ ]:
assert(not codebook_version in {4})

# nontraditional 2 (Level 35)
pd.set_option("display.max_colwidth", None)
display(da1[(da1["document"] == "2I1pDSUuKI.txt") & 
            (da1["conversation_number"] == 0) & 
            (da1["annotator"] == "A") & # output was too verbose
            (4100 < da1["quote_startPosition"]) & 
            (da1["quote_startPosition"] < 4616)])
pd.reset_option("display.max_colwidth")

### Helpers

#### Coding experience

In [ ]:
assert(not codebook_version in {4})

expda = da1[(da1["code_contentDomain"] == "codingExperience") & ~da1["speakerIsLearner"]]
expda = expda.sort_values(["document", "conversation_number", 
                           "quote_startPosition", "quote_endPosition", "annotator"])
expda = expda[["document", "conversation_number", "annotator", "quote_text", 
               "quote_startPosition", "quote_endPosition"]]

pd.set_option("display.max_colwidth", None)
expda.groupby(by=["document"]).apply(display)
pd.reset_option("display.max_colwidth")

#### Personal info

In [ ]:
assert(not codebook_version in {4})

perda = da1[(da1["code_contentDomain"] == "personalInfo") & ~da1["speakerIsLearner"]]
perda = perda.sort_values(["document", "conversation_number", 
                           "quote_startPosition", "quote_endPosition", "annotator"])
perda = perda[["document", "conversation_number", "annotator", "quote_text", 
               "quote_startPosition", "quote_endPosition"]]

pd.set_option("display.max_colwidth", None)
perda.groupby(by=["document"]).apply(display)
pd.reset_option("display.max_colwidth")

In [ ]:
assert(not codebook_version in {4})

# "I am in 11th grade"
pd.set_option("display.max_colwidth", None)
display(da1[(da1["document"] == "nvPpBOafGk.txt") & 
            (da1["conversation_number"] == 0) & 
            (da1["annotator"] == "A") & # output was too verbose
            #(2900 < da1["quote_startPosition"]) & 
            (da1["quote_startPosition"] < 3681)])
pd.reset_option("display.max_colwidth")

#### Teaching philosophy

In [ ]:
phida = da1[(da1["code"] == "Attitude, tone, or mood > expressTeachingPhilosophy") & 
            ~da1["speakerIsLearner"]]
phida = phida.sort_values(["document", "conversation_number", 
                           "quote_startPosition", "quote_endPosition", "annotator"])
phida = phida[["document", "conversation_number", "annotator", "quote_text", 
               "quote_startPosition", "quote_endPosition"]]

pd.set_option("display.max_colwidth", None)
phida.groupby(by=["document"]).apply(display)
pd.reset_option("display.max_colwidth")

## Attitude/tone/mood frequencies

In [ ]:
# get the relevant rows
attda = da1[da1["code"].str.startswith("Attitude, tone, or mood")].copy()
#attda["code"] = attda["code"].str[len("Attitude, tone, or mood > "):]
attda["code"] = attda["code"].str.split(" > ").str[-1]

# shorten some variable names to make the labels on the graph a little easier to look at
attda["code"] = attda["code"].replace(
    {"expressSatisfactionOrGratitude" : "satisfaction/gratitude", 
     "expressSupportingWords" : "supportingWords", 
     "expressTeachingPhilosophy" : "teachingPhilosophy", 
     "selfTalk" : "negativeSelfTalk"}) 

# compute the thing
def tmp(da):
    da = da["code"].value_counts()
    for c in attda["code"].unique():
        if not c in da.index:
            da[c] = 0
    return da.sort_index()

attfr = attda.groupby(by="annotator").apply(tmp)
attfr = attfr.median(axis=0).sort_values(ascending=False)

attfr

In [ ]:
# this is not particularly interesting imo
fig = plt.figure(figsize=(12, 4))
plt.bar(attfr.index, attfr)
fl, ft, fa = 18, 20, 20
plt.xlabel("Expression of attitude, tone, or mood", fontsize=fa)
plt.ylabel("Number of instances", fontsize=fa)
plt.title("Attitude/tone/mood frequencies across all data", fontsize=ft)
plt.xticks(rotation=50, ha="right", fontsize=fl)
fig.show()
if output_flag:
    fig.savefig(os.path.join(output_dir, "attitude-counts.png"), bbox_inches = "tight")

In [ ]:
attfr = attda.groupby(by=["voted_conversation_requests", "annotator"]).apply(tmp)
attfr = attfr.groupby(by="voted_conversation_requests").aggregate("median")
attfr

In [ ]:
for request in attfr.index:
    fr = attfr.loc[request]
    
    fig = plt.figure(figsize=(12, 4))
    plt.bar(fr.index, fr)
    fl, ft, fa = 18, 20, 20
    plt.xlabel("Expression of attitude, tone, or mood", fontsize=fa)
    plt.ylabel("Number of instances", fontsize=fa)
    plt.title("Attitude/tone/mood frequencies for request type {}".format(request), fontsize=ft)
    plt.xticks(rotation=50, ha="right", fontsize=fl)
    fig.show()
    if output_flag:
        fig.savefig(os.path.join(output_dir, "{}-attitude-counts.png".format(request)), bbox_inches = "tight")

In [ ]:
attfrp = attfr.div(attfr.sum(axis=1), axis=0)
attfrp = attfrp.rename(columns={"apology" : "Apology", 
                                "beingLost" : "Being lost", 
                                "beingWrong" : "Being wrong", 
                                "frustration" : "Frustration", 
                                "greeting" : "Greeting", 
                                "negativeSelfTalk" : "Negative self talk", 
                                "satisfaction/gratitude" : "Satisfaction or gratitude", 
                                "supportingWords" : "Supporting words", 
                                "teachingPhilosophy" : "Teaching philosophy"})
attfrp = attfrp.rename(index={"bugFix" : "Bug fix", 
                              "codeComprehension" : "Code comprehension", 
                              "codeImprove" : "Code improve", 
                              "codeWrite" : "Code write"})

In [ ]:
attfrp = attfrp[attfrp.sum().sort_values(ascending=False).index]
attfrp = attfrp.transpose()
attfrp

In [ ]:
ax = attfrp.plot(kind='bar', figsize=(12, 4), width=0.8)
ax.set_xlabel("Expression of attitude, tone, or mood", fontsize=fa)
ax.set_ylabel("Proportion of instances", fontsize=fa)
ax.set_title("Attitude/tone/mood frequencies broken down by request type", fontsize=ft)
ax.legend(fontsize=fl-2)
plt.grid()
plt.xticks(rotation=30, ha="right", fontsize=fl-2)
plt.yticks(fontsize=fl-2)
if output_flag:
    plt.savefig(os.path.join(output_dir, "request-attitude-counts.pdf"), bbox_inches = "tight")
else:
    plt.show()

- It's worth noting that `codeComprehension` and `codeImprove` request types both suffer from small denominators
- `greeting` is low for `codeComprehension` requests, potentially indicating that it's not usually the first request in a document
- `greeting` is high for `codeImprove` requests, potentially indicating that it doesn't tend to stem from prior requests
- `supportingWords` and `negativeSelfTalk` are both high for `codeComprehension` requests, potentially indicating that Learners find this more difficult than other request types
- `supportingWords` is low for `codeImprove` requests, potentially indicating that Helpers perceive that Learners are struggling less or at a more advanced level

### Taking a look at the conversation number hypothesis

In [ ]:
attfr = attda.groupby(by=["document", "conversation_number", "annotator"]).apply(tmp)
attfr = attfr.groupby(by=["document", "conversation_number"]).aggregate("median")
attfr = attfr.groupby(by="conversation_number").aggregate("mean")
attfr

In [ ]:
for n in attfr.index:
    fr = attfr.loc[n]
    
    fig = plt.figure(figsize=(12, 4))
    plt.bar(fr.index, fr)
    fl, ft, fa = 18, 20, 20
    plt.xlabel("Expression of attitude, tone, or mood", fontsize=fa)
    plt.ylabel("Average number of instances", fontsize=fa)
    plt.title("Attitude/tone/mood frequencies for conversation {}".format(n), fontsize=ft)
    plt.xticks(rotation=50, ha="right", fontsize=fl)
    fig.show()
    if output_flag:
        fig.savefig(os.path.join(output_dir, "conv{}-attitude-counts.png".format(n)), bbox_inches = "tight")

In [ ]:
ax = attfr.transpose().plot(kind='bar', figsize=(12, 4), width=0.8)
ax.set_xlabel("Expression of attitude, tone, or mood", fontsize=fa)
ax.set_ylabel("Average number of instances", fontsize=fa)
ax.set_title("Attitude/tone/mood frequencies broken down by conversation number", fontsize=ft)
ax.legend(fontsize=fl-2)
plt.xticks(rotation=50, ha="right", fontsize=fl)
if output_flag:
    plt.savefig(os.path.join(output_dir, "conv-attitude-counts.png"), bbox_inches = "tight")
else:
    plt.show()

In [ ]:
# interaction 3 didn't turn out well lol

In [ ]:
# FIXME do it again but replace attitude with request type